# Cleaning Data

In [11]:
we are now switching to a higher level api:

Dataframes

they are build upon RDD but will make our life much simpleer

They will inherit some of the greate attributes of RDD:

 * in memory
 * partitioned
 * resilient
 * read only

Treats the Data similar like relational databases:

Set of Rows where each row has columns of certain types


Dataframes

will alow spark to apply optimazations that plain rdd will now allow

this leads to less development effort

current an future developtment of spark is mostly on dataframe apis

when you have no proper reason, default to dataframes


Optimizations on build time
by catalyst optimizer for query optimization

and with tungsten on runtime


SyntaxError: invalid syntax (991598742.py, line 1)

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("dataframes").master("local[4]").getOrCreate()

In [13]:
spark

In [24]:
from IPython.display import *
display(HTML("<style>pre { white-space: pre !important; }</style>"))

![alt text](../../slides/history-of-spark.drawio.svg "Spark Optimization Engines")

## Catalyst Optimizer

Query Optimer to produce optimued query plan

## Tungsten Engine

Imporoves efficizeniez ddruing 
P

Phases

* Analysis
* logigcal Optimzzation
* Pyhsical Planning
* Code Generation
* 



### One Example Process


1. Analysis Phase
2. Create an unresolved logical plan
   Just syntax errors. no data types checks
   of if the exist att all
3. Than catalyst looks up the catalog for columns and its types
4. Than it creates the logical plan and run logical optimization
   it applies rules, constant folding, predicate pushdown
5. this results in the optimized logical plan
6. than the pysical plan
7. then a costmodel is applied
8. the we have a concrete pyshical plan
9. rdd code is being generated
10. 

keep in mind that rdd code does  not get any of these optimizations!



show performance comparisoin  scala python rdd, dataframe


# Wie kannst Du Dataframes erzeugen

* aus einem RDD oder einer beliebigen Python-Collections
* aus anderen Dataframes
* aus Dateien/Datenbanken

In [23]:
spark = (
    SparkSession
    .builder
    .appName("TaxiOperationsDataFrameApp")
    .master("local[4]")
    .config("spark.dynamicAllocation.enabled", "false")
    .config("spark.sql.adaptive.enabled", "false")
    .getOrCreate()
)
sc = spark.sparkContext
spark

### Erzeuge ein Dataframe aus einem RDD

In [25]:
data = [
    [1, "Oliver", 10],
    [2, "Xiaofei", 20],
    [3, "Marc", 40]
]

dozenten_rdd = sc.parallelize(data)

In [26]:
dozenten_df = dozenten_rdd.toDF()

In [27]:
dozenten_df.show()

+---+-------+---+
| _1|     _2| _3|
+---+-------+---+
|  1| Oliver| 10|
|  2|Xiaofei| 20|
|  3|   Marc| 40|
+---+-------+---+



In [28]:
dozenten_df = dozenten_df.toDF("ID", "Name", "Points")

In [29]:
dozenten_df.show()

+---+-------+------+
| ID|   Name|Points|
+---+-------+------+
|  1| Oliver|    10|
|  2|Xiaofei|    20|
|  3|   Marc|    40|
+---+-------+------+



In [30]:
dozenten_df.printSchema()

root
 |-- ID: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Points: long (nullable = true)



### Erzeuge Dataframe aus einer Collection

In [39]:
dozenten_df = (
    spark.createDataFrame(
        data,
        "Id: long, Name: string, Pointy: long"
    )
)

In [34]:
dozenten_df.show()

+---+-------+------+
| Id|   Name|Pointy|
+---+-------+------+
|  1| Oliver|    10|
|  2|Xiaofei|    20|
|  3|   Marc|    40|
+---+-------+------+



In [36]:
dozenten_df.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Pointy: string (nullable = true)



In [40]:
type(spark)

pyspark.sql.session.SparkSession

### Erzeuge ein Dataframe aus einer Datei

In [41]:
yellow_taxis_df = spark.read.csv("YellowTaxis_202210.csv", header=True)

In [43]:
yellow_taxis_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1|2022-10-01T05:33:...| 2022-10-01T05:48:...|            1.0|          1.7|       1.0|                 N|         249|         107|           1|        9.5|  3.0|    0.5|      2.6

In [42]:
yellow_taxis_df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)
 |-- airport_fee: string (nullable = true)



In [45]:
green_taxi_df = (
    spark.read.option("header", "true").option("delimiter", "\t").csv("GreenTaxis_202210.csv")
)

In [46]:
green_taxi_df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorId|lpep_pickup_datetime|lpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2|2022-10-01T06:08:...| 2022-10-01T06:21:...|            1.0|         2.47|       1.0|                 N|         256|         225|         1.0|       11.5|  0.5|    0.5|      2.5

In [47]:
paymen_types_df = (
    spark.read.json("PaymentTypes.json")
)

In [49]:
paymen_types_df.show()

+-----------+-------------+
|PaymentType|PaymentTypeID|
+-----------+-------------+
|Credit Card|            1|
|       Cash|            2|
|  No Charge|            3|
|    Dispute|            4|
|    Unknown|            5|
|Voided Trip|            6|
+-----------+-------------+



In [50]:
less

UsageError: Missing filename.


In [51]:
ls

 dataframes.ipynb           PaymentTypes.json        YellowTaxis_202210.zip
 GreenTaxis_202210.csv      quickstart_df.ipynb
'm5.1 - DataFrames.ipynb'   YellowTaxis_202210.csv


In [52]:
cat PaymentTypes.json

{"PaymentTypeID":1,"PaymentType":"Credit Card"}
{"PaymentTypeID":2,"PaymentType":"Cash"}
{"PaymentTypeID":3,"PaymentType":"No Charge"}
{"PaymentTypeID":4,"PaymentType":"Dispute"}
{"PaymentTypeID":5,"PaymentType":"Unknown"}
{"PaymentTypeID":6,"PaymentType":"Voided Trip"}


In [53]:
less PaymentTypes.json

{"PaymentTypeID":1,"PaymentType":"Credit Card"}
{"PaymentTypeID":2,"PaymentType":"Cash"}
{"PaymentTypeID":3,"PaymentType":"No Charge"}
{"PaymentTypeID":4,"PaymentType":"Dispute"}
{"PaymentTypeID":5,"PaymentType":"Unknown"}
{"PaymentTypeID":6,"PaymentType":"Voided Trip"}


In [65]:
yellow_taxis_df = (
    spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("YellowTaxis_202210.csv")
)

In [66]:
yellow_taxis_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)

